# Module for Sentinel 1 Orbit Data Acquisition

## Why Orbit Data Are Required in InSAR / DInSAR

Orbit data are needed to accurately describe the satellite’s position and velocity at acquisition time. They are used to compute the imaging geometry, baseline between acquisitions, and phase components related to range and topography. Without accurate orbit information, co-registration, interferogram formation, and deformation estimation become biased or unstable.

---

## Precise Orbit vs Restituted (Rapid) Orbit

### Precise Orbit (POEORB)

* Highest accuracy orbit solution
* Computed after full tracking data are collected and refined
* Typical accuracy: **centimeter-level**
* Best for:

  * DInSAR deformation analysis
  * Time-series InSAR (SBAS / PS)
  * Baseline-sensitive processing
* Availability: delayed (usually days to weeks after acquisition)

---

### Restituted / Rapid Orbit (RESORB)

* Quickly generated preliminary orbit
* Uses predicted + partial tracking data
* Typical accuracy: **decimeter to meter-level**
* Best for:

  * Quick-look processing
  * Near–real-time applications
  * Initial tests and QC runs
* Availability: fast (within hours)

---

## Practical Rule

* Final scientific DInSAR products → **use Precise Orbit**
* Quick preview / pipeline testing → **Restituted Orbit is acceptable**


## 0. Pre-Setup

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

# get current working dir
%pwd

'/raven/u/yhsuh/coding/INSAR/examples'

In [2]:
import glob
import os
import pandas as pd
from modules.orbit_manager import OrbitManager

# 1. 실제 파일 목록 가져오기
RAW_DATA_DIR = "raw_data"
real_files = glob.glob(os.path.join(RAW_DATA_DIR, "*.zip"))

# 2. OrbitManager 초기화 및 실행
orbit_man = OrbitManager(orbit_dir="aux_orbits")

print(f"\n🚀 [Start] {len(real_files)}개의 파일에 대해 궤도 다운로드 시작...")
df_report = orbit_man.fetch_orbits(real_files, precise_only=False)

# 3. 결과 확인
try:
    display(df_report.style.set_properties(**{'text-align': 'left'}))
except:
    print(df_report)

[Auth] Initializing NASA Earthdata Authentication...
 [Auth] Logged in successfully.

🚀 [Start] 2개의 파일에 대해 궤도 다운로드 시작...

[OrbitManager] Checking orbits for 2 scenes...
               Target Dir: aux_orbits
               Mode: Auto Fallback (POE -> RES)


,Scene ID,Acq Date,Orbit Type,Status
0,S1C_IW_SLC__1SDV_20251221T213001_20251221T213028_005555_00B135_DA0F.zip,20251221,Precise (POE),✅ Success
1,S1C_IW_SLC__1SDV_20260102T213001_20260102T213028_005730_00B73A_280A.zip,20260102,Precise (POE),✅ Success


In [3]:
orbit_man.get_status()

{'module_name': 'OrbitManager',
 'is_ready': True,
 'orbit_dir': '/raven/u/yhsuh/coding/INSAR/examples/aux_orbits',
 'aux_dir': '/raven/u/yhsuh/coding/INSAR/examples/aux_orbits',
 'file_count': 2}

In [4]:
import os
import shutil
import pandas as pd
from pathlib import Path
from modules.orbit_manager import OrbitManager

# ==============================================================================
# [Setup] 테스트 환경 설정
# ==============================================================================
MOCK_SLC_DIR = "test_mock_slc_full"
TEST_ORBIT_DIR = "test_orbits_full"

# 깨끗한 상태에서 시작
for d in [MOCK_SLC_DIR, TEST_ORBIT_DIR]:
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

# ------------------------------------------------------------------------------
# 가짜 SLC 파일 생성 (다양한 시나리오용)
# ------------------------------------------------------------------------------
# 1. [S1A Past] 2021년 데이터 -> POE (정밀 궤도)가 확실히 존재함
file_s1a_poe = "S1A_IW_SLC__1SDV_20210501T060000_20210501T060030_037690_047240_1234.zip"

# 2. [S1C Recent] 2026년 데이터 -> RES (준정밀)만 존재한다고 가정 (테스트용)
file_s1c_res = "S1C_IW_SLC__1SDV_20260201T000000_20260201T000030_001000_00ABCD_5678.zip"

# 3. [Invalid] 형식이 잘못된 파일
file_invalid = "NOT_A_VALID_SENTINEL_FILE.zip"

mock_files_map = {
    "POE_CASE": os.path.join(MOCK_SLC_DIR, file_s1a_poe),
    "RES_CASE": os.path.join(MOCK_SLC_DIR, file_s1c_res),
    "BAD_CASE": os.path.join(MOCK_SLC_DIR, file_invalid)
}

print(f"🔨 가짜 SLC 파일 생성 중...")
for key, path in mock_files_map.items():
    Path(path).touch()
    print(f"   Created [{key}]: {Path(path).name}")


# ==============================================================================
# [Test 1] 초기화 및 get_status (Empty State)
# ==============================================================================
print("\n🔵 [Test 1] 초기화 및 get_status (Empty Check)")
orb_man = OrbitManager(orbit_dir=TEST_ORBIT_DIR)
status = orb_man.get_status()

print(f"   Status: {status}")
if status['file_count'] == 0 and status['is_ready'] is True:
    print("✅ 초기 상태 검증 완료.")
else:
    print("❌ 초기 상태 이상.")

# ==============================================================================
# [Test 2] Normal Mode (precise_only=False) - Auto Fallback 테스트
# ==============================================================================
print("\n🔵 [Test 2] Normal Mode: Auto Fallback (POE -> RES)")
# 입력: POE 케이스, RES 케이스, 잘못된 파일
inputs = [mock_files_map["POE_CASE"], mock_files_map["RES_CASE"], mock_files_map["BAD_CASE"]]

df_normal = orb_man.fetch_orbits(inputs, precise_only=False)

print("\n📊 Normal Mode Result:")
display(df_normal[['Scene ID', 'Orbit Type', 'Status']])

# 검증 로직
chk_poe = df_normal[df_normal['Scene ID'] == file_s1a_poe]['Orbit Type'].iloc[0]
chk_res = df_normal[df_normal['Scene ID'] == file_s1c_res]['Orbit Type'].iloc[0]
chk_bad = df_normal[df_normal['Scene ID'] == file_invalid]['Status'].iloc[0]

# 1. S1A는 POE여야 함
if "Precise" in chk_poe:
    print("✅ S1A (Past): Precise Orbit (POE) 다운로드 성공.")
else:
    print(f"❌ S1A 결과 이상: {chk_poe}")

# 2. S1C는 RES 또는 Fallback이어야 함 (precise_only=False이므로 성공해야 함)
if "Restituted" in chk_res or "Precise" in chk_res:
    print(f"✅ S1C (Recent): 궤도 파일 확보 성공 ({chk_res}).")
else:
    print(f"❌ S1C 실패: {chk_res}")

# 3. 잘못된 파일은 에러 처리되어야 함
if "Invalid" in chk_bad:
    print("✅ Invalid File: 적절히 필터링됨.")
else:
    print(f"❌ Invalid File 처리 실패: {chk_bad}")

# 상태 업데이트 확인
status_after = orb_man.get_status()
print(f"   Updated Status: File Count = {status_after['file_count']}")


# ==============================================================================
# [Test 3] Strict Mode (precise_only=True) - 실패 유도 테스트
# ==============================================================================
print("\n🔵 [Test 3] Strict Mode: RESORB Fail Check")
# 테스트를 위해 폴더 초기화 (다운로드된 파일 삭제)
shutil.rmtree(TEST_ORBIT_DIR)
orb_man_strict = OrbitManager(orbit_dir=TEST_ORBIT_DIR)

# S1C (RES만 있는 경우)를 Strict Mode로 요청 -> 실패해야 정상
df_strict = orb_man_strict.fetch_orbits([mock_files_map["RES_CASE"]], precise_only=True)

print("\n📊 Strict Mode Result:")
display(df_strict[['Scene ID', 'Orbit Type', 'Status']])

row = df_strict.iloc[0]
if "Restituted" in row['Orbit Type'] and "Failed" in row['Status']:
    print("✅ Strict Mode 검증 성공: RESORB 파일을 찾았으나 정책에 의해 실패 처리함.")
elif "Precise" in row['Orbit Type']:
    print("⚠️ 주의: 해당 날짜의 정밀 궤도(POE)가 출시되어 성공해버렸습니다. (테스트 날짜 조정 필요)")
else:
    print(f"❌ Strict Mode 동작 실패: {row['Status']}")


# ==============================================================================
# [Test 4] Strict Mode - 성공 케이스 테스트
# ==============================================================================
print("\n🔵 [Test 4] Strict Mode: POE Success Check")
# S1A (POE 있는 경우)를 Strict Mode로 요청 -> 성공해야 정상
df_strict_pass = orb_man_strict.fetch_orbits([mock_files_map["POE_CASE"]], precise_only=True)

print("\n📊 Strict Mode Success Check:")
display(df_strict_pass[['Scene ID', 'Orbit Type', 'Status']])

row_pass = df_strict_pass.iloc[0]
if "Precise" in row_pass['Orbit Type'] and "Success" in row_pass['Status']:
    print("✅ Strict Mode 검증 성공: 정밀 궤도(POE)가 있어 정상적으로 다운로드됨.")
else:
    print(f"❌ Strict Mode 실패: {row_pass['Status']}")

print("\n🎉 모든 통합 테스트 완료.")

🔨 가짜 SLC 파일 생성 중...
   Created [POE_CASE]: S1A_IW_SLC__1SDV_20210501T060000_20210501T060030_037690_047240_1234.zip
   Created [RES_CASE]: S1C_IW_SLC__1SDV_20260201T000000_20260201T000030_001000_00ABCD_5678.zip
   Created [BAD_CASE]: NOT_A_VALID_SENTINEL_FILE.zip

🔵 [Test 1] 초기화 및 get_status (Empty Check)
[Auth] Initializing NASA Earthdata Authentication...
 [Auth] Logged in successfully.
   Status: {'module_name': 'OrbitManager', 'is_ready': True, 'orbit_dir': '/raven/u/yhsuh/coding/INSAR/examples/test_orbits_full', 'aux_dir': '/raven/u/yhsuh/coding/INSAR/examples/test_orbits_full', 'file_count': 0}
✅ 초기 상태 검증 완료.

🔵 [Test 2] Normal Mode: Auto Fallback (POE -> RES)

[OrbitManager] Checking orbits for 3 scenes...
               Target Dir: test_orbits_full
               Mode: Auto Fallback (POE -> RES)

📊 Normal Mode Result:


,Scene ID,Orbit Type,Status
0,NOT_A_VALID_SENTINEL_FILE.zip,-,❌ Invalid Filename
1,S1A_IW_SLC__1SDV_20210501T060000_20210501T0600...,Precise (POE),✅ Success
2,S1C_IW_SLC__1SDV_20260201T000000_20260201T0000...,Restituted (RES),✅ Success (RES Found)


✅ S1A (Past): Precise Orbit (POE) 다운로드 성공.
✅ S1C (Recent): 궤도 파일 확보 성공 (Restituted (RES)).
✅ Invalid File: 적절히 필터링됨.
   Updated Status: File Count = 2

🔵 [Test 3] Strict Mode: RESORB Fail Check
[Auth] Initializing NASA Earthdata Authentication...
 [Auth] Logged in successfully.

[OrbitManager] Checking orbits for 1 scenes...
               Target Dir: test_orbits_full
               Mode: Strict (Precise Only)

📊 Strict Mode Result:


,Scene ID,Orbit Type,Status
0,S1C_IW_SLC__1SDV_20260201T000000_20260201T0000...,Restituted (RES),❌ Failed (Strict Mode: Only RES found)


✅ Strict Mode 검증 성공: RESORB 파일을 찾았으나 정책에 의해 실패 처리함.

🔵 [Test 4] Strict Mode: POE Success Check

[OrbitManager] Checking orbits for 1 scenes...
               Target Dir: test_orbits_full
               Mode: Strict (Precise Only)

📊 Strict Mode Success Check:


,Scene ID,Orbit Type,Status
0,S1A_IW_SLC__1SDV_20210501T060000_20210501T0600...,Precise (POE),✅ Success


✅ Strict Mode 검증 성공: 정밀 궤도(POE)가 있어 정상적으로 다운로드됨.

🎉 모든 통합 테스트 완료.
